In [1]:
# general imports
import numpy as np
import os

#### Fragen
- nur auf eine trial datei beziehen immer oder auf alle?
- wie soll ich mit den Perosnen bezogenen angaben umgehen welche benötigt werden?: read_file (line 104)
- Condition: Left foot strikes first on the ground !!!!!!!Needs to be adapted for multiple Gait-Cycles!!!!!!! - Kati fragen, wie das beschrieben werden soll für unterschiedliche Beinzyklen: read_file (line 85)
- in uitls.force.norm_gc_para: upper bounds and lower bounds for everyone the same? - weiß si ezufällig die Literatur zu den Literaturwerten?
- in utils.marker.emg_data emg_L_roh and emg_R_roh really the same?

## Prepairing Data

### Loading Data from c3d files

In [2]:
# Import functions for reading and preparing c3d file data to panda dataframes
import read_file as rmd

In [3]:
# Adapt path to dynamic and static c3d file
pc =  r'C:\UNI\HIWI\TUM\HIWI_SPM\Messdaten\Masterarbeit Data\Data final\Christian\Gait FullBody'

c3d_file = os.path.join(pc, 'dyn01.c3d')
# path c3d file with static data
c3d_file_stat = os.path.join(pc, 'stat01.c3d')

In [4]:
# read marker and force data from c3d file
FB_mark = rmd.read_marker_data(c3d_file, c3d_file_stat)
FB_force = rmd.read_force_data(c3d_file, c3d_file_stat)

Analysis data found in c3d file


FutureWarning [c:\UNI\HIWI\TUM\HIWI_SPM\GIND_Repo\The-Gait-Index-for-Neurological-Disorders-GIND-\utils\marker\emg_data.py:86] Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
FutureWarning [c:\UNI\HIWI\TUM\HIWI_SPM\GIND_Repo\The-Gait-Index-for-Neurological-Disorders-GIND-\utils\marker\emg_data.py:101] Series.__setitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To set a value by position, use `ser.iloc[pos] = value`


Analysis data found in c3d file




### Loading earlier calculated variables

In [5]:
# include health values for reference and calculations (already calculated from 3 healthy subjects)
cop_healthy_m = np.load('utils/variables/cop_healthy_m.npy')
g_healthy_m = np.load('utils/variables/g_healthy_m.npy')
grf_healthy_m = np.load('utils/variables/grf_healthy_m.npy')
GVS_healthy = np.load('utils/variables/GVS_healthy.npy')
GVS_cop_healthy = np.load('utils/variables/GVS_cop_healthy.npy')
GVS_grf_healthy = np.load('utils/variables/GVS_grf_healthy.npy')

## Calculating GIND

### Calcuation of GVS?

In [6]:
# Parameters for calculatio of gait symmetry
para = np.array([12, 13, 14, 9, 10, 11, 6, 3, 17]) - 1

# array inits for arrays in line 
row_MW_R= FB_mark["gc_count_mark"]["R_num_gc_ges"] 
row_MW_L= FB_mark["gc_count_mark"]["L_num_gc_ges"]

angles_ges_mean = np.zeros(51)
g_l_healthy = np.zeros(len(para)*angles_ges_mean.shape[0])
g_r_healthy = np.zeros(len(para)*angles_ges_mean.shape[0])

# loop for all the parameters that need to be collected
for n in range(len(para)):
    # calculate g_l for left side
    angles_red_ges = np.zeros((51, row_MW_L), dtype=np.float64)
    for k in range(0, row_MW_L):
        angles_curr = FB_mark["L_gc_ang_ges_MW"].iloc[k, para[n]]
        angles_red = angles_curr[0:201:4]
        angles_red_ges[:, k] = angles_red
        angles_curr = []
        angles_red = []
    angles_ges_mean[:] = np.mean(angles_red_ges[:, 0:row_MW_L], axis=1)
    g_l_healthy[n*angles_ges_mean.shape[0]:(n+1)*angles_ges_mean.shape[0]] = angles_ges_mean;
    
    # calculate right marks
    angles_red_ges = np.zeros((51, row_MW_R), dtype=np.float64)
    for k in range(0, row_MW_R):
        angles_curr = FB_mark["R_gc_ang_ges_MW"].iloc[k, para[n]]
        angles_red = angles_curr[0:201:4]
        angles_red_ges[:, k] = angles_red
    angles_ges_mean[:] = np.mean(angles_red_ges[:, 0:row_MW_R], axis=1)
    g_r_healthy[n*angles_ges_mean.shape[0]:(n+1)*angles_ges_mean.shape[0]] = angles_ges_mean

In [7]:
p = 9 # number of parameters (len(para))
GVS_l_healthy = np.array([np.sqrt(np.mean(np.square(g_l_healthy[i*51:(i+1)*51]-g_healthy_m[i*51:(i+1)*51]))) for i in range(p)])
GVS_r_healthy = np.array([np.sqrt(np.mean(np.square(g_r_healthy[i*51:(i+1)*51]-g_healthy_m[i*51:(i+1)*51]))) for i in range(p)])

# Final values for GIND calculation
V_GVS_l_healthy = GVS_l_healthy / GVS_healthy
V_GVS_r_healthy = GVS_r_healthy / GVS_healthy

### Calculation of GVS GRF

In [8]:
para = np.array([6, 7, 8]) - 1

row_MW_R = FB_force["gc_count_force"]["gz_counter_R_ges"]
row_MW_L = FB_force["gc_count_force"]["gz_counter_L_ges"]

grf_ges_mean = np.zeros(51)
grf_l_healthy = np.zeros(len(para)*grf_ges_mean.shape[0])
grf_r_healthy = np.zeros(len(para)*grf_ges_mean.shape[0])

for n in range(len(para)):
    grf_red_ges = np.zeros((51, row_MW_L), dtype=np.float64)
    for k in range(row_MW_L):
        grf_curr = FB_force["L_gc_grf_ges_MW"].iloc[k, para[n]]
        grf_red = grf_curr[0:201:4].T
        grf_red_ges[:, k] = grf_red
        grf_curr = []
        grf_red = []
    grf_ges_mean[:] = np.mean(grf_red_ges[:, 0:row_MW_L], axis=1)
    grf_l_healthy[n*grf_ges_mean.shape[0]:(n+1)*grf_ges_mean.shape[0]] = grf_ges_mean

    grf_ges_mean = np.zeros(51)
    grf_red_ges = np.zeros((51, row_MW_R), dtype=np.float64)
    for k in range(row_MW_R):
        grf_curr = FB_force["R_gc_grf_ges_MW"].iloc[k, para[n]]
        grf_red = grf_curr[0:201:4].T
        grf_red_ges[:, k] = grf_red
        grf_curr = []
        grf_red = []
    grf_ges_mean[:] = np.mean(grf_red_ges[:, 0:row_MW_R], axis=1)
    grf_r_healthy[n*grf_ges_mean.shape[0]:(n+1)*grf_ges_mean.shape[0]] = grf_ges_mean

In [9]:
p = 3 # len(para)
GVS_grf_l_healthy = np.array([np.sqrt(np.mean(np.square(grf_l_healthy[i*51:(i+1)*51]-grf_healthy_m[i*51:(i+1)*51]))) for i in range(p)])
GVS_grf_r_healthy = np.array([np.sqrt(np.mean(np.square(grf_r_healthy[i*51:(i+1)*51]-grf_healthy_m[i*51:(i+1)*51]))) for i in range(p)])

V_GVS_grf_l_healthy = GVS_grf_l_healthy / GVS_grf_healthy
V_GVS_grf_r_healthy = GVS_grf_r_healthy / GVS_grf_healthy

### Calculation of GVS COP

In [10]:
para = np.array([3, 4, 6, 7, 9, 10]) -1

row_MW_R = FB_force["gc_count_force"]["gz_counter_R_ges"]
row_MW_L = FB_force["gc_count_force"]["gz_counter_L_ges"]
cop_ges_mean = np.zeros(51)
cop_l_healthy = np.zeros(len(para)*cop_ges_mean.shape[0])
cop_r_healthy = np.zeros(len(para)*cop_ges_mean.shape[0])

for n in range(len(para)):
    cop_red_ges = np.zeros((51, row_MW_L), dtype=np.float64)
    for k in range(0, row_MW_L):
        cop_curr = FB_force["L_gc_cop_ges_MW"].iloc[k, para[n]]
        cop_red = cop_curr[0:201:4].T
        cop_red_ges[:, k] = cop_red
        cop_curr = []
        cop_red = []
    cop_ges_mean[:] = np.mean(cop_red_ges[:, 0:row_MW_L], axis=1)
    cop_l_healthy[n*cop_ges_mean.shape[0]:(n+1)*cop_ges_mean.shape[0]] = cop_ges_mean
    
    cop_red_ges = np.zeros((51, row_MW_R), dtype=np.float64)
    for k in range(0, row_MW_R):
        cop_curr = FB_force["R_gc_cop_ges_MW"].iloc[k, para[n]]
        cop_red = cop_curr[0:201:4].T
        cop_red_ges[:, k] = cop_red
        cop_curr = []
        cop_red = []
    cop_ges_mean[:] = np.mean(cop_red_ges[:, 0:row_MW_L], axis=1)
    cop_r_healthy[n*cop_ges_mean.shape[0]:(n+1)*cop_ges_mean.shape[0]] = cop_ges_mean

In [11]:
p = 6 # (len(p))
GVS_cop_l_healthy = np.array([np.sqrt(np.mean(np.square(cop_l_healthy[i*51:(i+1)*51]-cop_healthy_m[i*51:(i+1)*51]))) for i in range(p)])
GVS_cop_r_healthy = np.array([np.sqrt(np.mean(np.square(cop_r_healthy[i*51:(i+1)*51]-cop_healthy_m[i*51:(i+1)*51]))) for i in range(p)])

V_GVS_cop_l_healthy = GVS_cop_l_healthy / GVS_cop_healthy
V_GVS_cop_r_healthy = GVS_cop_r_healthy / GVS_cop_healthy

### Summarizing all calculations for the GIND

In [12]:
def rms(values):
    return np.sqrt(np.mean(np.square(values)))

In [13]:
V_GPS_gesamt_l_healthy= rms([V_GVS_l_healthy[0], V_GVS_l_healthy[3], V_GVS_l_healthy[6], V_GVS_l_healthy[7], V_GVS_grf_l_healthy[2], V_GVS_cop_l_healthy[1]])
V_GPS_gesamt_r_healthy= rms([V_GVS_r_healthy[0], V_GVS_r_healthy[3], V_GVS_r_healthy[6], V_GVS_r_healthy[7], V_GVS_grf_r_healthy[2], V_GVS_cop_r_healthy[1]])

In [14]:
V_GPS_gesamt_r_healthy

1.4555576966825823

In [15]:
V_GPS_gesamt_o_healthy_c = rms([
    V_GVS_l_healthy[0],  # 1 in MATLAB
    V_GVS_l_healthy[3],  # 4 in MATLAB
    V_GVS_l_healthy[6],  # 7 in MATLAB
    V_GVS_l_healthy[7],  # 8 in MATLAB
    V_GVS_grf_l_healthy[2],  # 3 in MATLAB
    V_GVS_cop_l_healthy[1],  # 2 in MATLAB
    V_GVS_r_healthy[3],  # 4 in MATLAB
    V_GVS_r_healthy[6],  # 7 in MATLAB
    V_GVS_r_healthy[7],  # 8 in MATLAB
    V_GVS_grf_r_healthy[2],  # 3 in MATLAB
    V_GVS_cop_r_healthy[1]   # 2 in MATLAB
])

In [16]:
V_GPS_gesamt_o_healthy_c

1.1828959344072307